In [1]:
import numpy as np
import glob as glob
from os.path import join, basename, dirname, exists
import sys
print(sys.executable)  #print kernel path
#print(sys.path)

# tensorflow imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import models
from tensorflow.keras import layers
print(tf.__version__)

# custom imports
import librispect as lspct
from librispect.features import predict
from librispect.utils import split_validation

from data_utils import SortedNumberGenerator


/home/AD/kachiem/miniconda3/envs/tf-conda/bin/python3
1.13.1


attempting to connect srihita's spectrogram function to train_model

In [13]:
def network_encoder(x, code_size):

    ''' Define the network mapping images to embeddings '''

    x = keras.layers.Conv2D(filters=64, kernel_size=3, strides=2, activation='linear')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.LeakyReLU()(x)
    x = keras.layers.Conv2D(filters=64, kernel_size=3, strides=2, activation='linear')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.LeakyReLU()(x)
    x = keras.layers.Conv2D(filters=64, kernel_size=3, strides=2, activation='linear')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.LeakyReLU()(x)
    x = keras.layers.Conv2D(filters=64, kernel_size=3, strides=2, activation='linear')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.LeakyReLU()(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(units=256, activation='linear')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.LeakyReLU()(x)
    x = keras.layers.Dense(units=code_size, activation='linear', name='encoder_embedding')(x)

    return x


def network_autoregressive(x):

    ''' Define the network that integrates information along the sequence '''

    # x = keras.layers.GRU(units=256, return_sequences=True)(x)
    # x = keras.layers.BatchNormalization()(x)
    x = keras.layers.GRU(units=256, return_sequences=False, name='ar_context')(x)

    return x


def network_prediction(context, code_size, predict_terms):

    ''' Define the network mapping context to multiple embeddings '''

    outputs = []
    for i in range(predict_terms):
        outputs.append(keras.layers.Dense(units=code_size, activation="linear", name='z_t_{i}'.format(i=i))(context))

    if len(outputs) == 1:
        output = keras.layers.Lambda(lambda x: K.expand_dims(x, axis=1))(outputs[0])
    else:
        output = keras.layers.Lambda(lambda x: K.stack(x, axis=1))(outputs)

    return output


class CPCLayer(keras.layers.Layer):

    ''' Computes dot product between true and predicted embedding vectors '''

    def __init__(self, **kwargs):
        super(CPCLayer, self).__init__(**kwargs)

    def call(self, inputs):

        # Compute dot product among vectors
        preds, y_encoded = inputs
        dot_product = K.mean(y_encoded * preds, axis=-1)
        dot_product = K.mean(dot_product, axis=-1, keepdims=True)  # average along the temporal dimension

        # Keras loss functions take probabilities
        dot_product_probs = K.sigmoid(dot_product)

        return dot_product_probs

    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], 1)


def network_cpc(image_shape, terms, predict_terms, code_size, learning_rate):

    ''' Define the CPC network combining encoder and autoregressive model '''

    # Set learning phase (https://stackoverflow.com/questions/42969779/keras-error-you-must-feed-a-value-for-placeholder-tensor-bidirectional-1-keras)
    K.set_learning_phase(1)

    # Define encoder model
    encoder_input = keras.layers.Input(image_shape)
    encoder_output = network_encoder(encoder_input, code_size)
    encoder_model = keras.models.Model(encoder_input, encoder_output, name='encoder')
    encoder_model.summary()

    # Define rest of model
    x_input = keras.layers.Input((terms, image_shape[0], image_shape[1], image_shape[2]))
    x_encoded = keras.layers.TimeDistributed(encoder_model)(x_input)
    context = network_autoregressive(x_encoded)
    preds = network_prediction(context, code_size, predict_terms)

    y_input = keras.layers.Input((predict_terms, image_shape[0], image_shape[1], image_shape[2]))
    y_encoded = keras.layers.TimeDistributed(encoder_model)(y_input)

    # Loss
    dot_product_probs = CPCLayer()([preds, y_encoded])

    # Model
    cpc_model = keras.models.Model(inputs=[x_input, y_input], outputs=dot_product_probs)

    # Compile model
    cpc_model.compile(
        optimizer=keras.optimizers.Adam(lr=learning_rate),
        loss='binary_crossentropy',
        metrics=['binary_accuracy']
    )
    cpc_model.summary()

    return cpc_model




In [2]:
def train_model(epochs, batch_size, output_dir, code_size, lr=1e-4, terms=4, predict_terms=4, image_size=28, color=False):

    # Prepare data
    train_data = SortedNumberGenerator(batch_size=batch_size, subset='train', terms=terms,
                                       positive_samples=batch_size // 2, predict_terms=predict_terms,
                                       image_size=image_size, color=color, rescale=True)

    validation_data = SortedNumberGenerator(batch_size=batch_size, subset='valid', terms=terms,
                                            positive_samples=batch_size // 2, predict_terms=predict_terms,
                                            image_size=image_size, color=color, rescale=True)

    # Prepares the model
    model = network_cpc(image_shape=(image_size, image_size, 3), terms=terms, predict_terms=predict_terms,
                        code_size=code_size, learning_rate=lr)

    # Callbacks
    callbacks = [keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=1/3, patience=2, min_lr=1e-4)]

    # Trains the model
    model.fit_generator(
        generator=train_data,
        steps_per_epoch=len(train_data),
        validation_data=validation_data,
        validation_steps=len(validation_data),
        epochs=epochs,
        verbose=1,
        callbacks=callbacks
    )

    # Saves the model
    # Remember to add custom_objects={'CPCLayer': CPCLayer} to load_model when loading from disk
    model.save(join(output_dir, 'cpc.h5'))

    # Saves the encoder alone
    encoder = model.layers[1].layer
    encoder.save(join(output_dir, 'encoder.h5'))


shaping input for spectrogram functions

In [4]:
'''
Instead of images and labels, I have sectrogram segments x and y.
So x is input and y would be output (aka labels). 
You train on bunch of (x,y), let's call it train_x and train_y.
Then when you input test_x, you should get test_y (or vis_x and vis_y in this case)
'''

# spect_height = number of freq. bins
# y always has one time bin
# n_lags is number of time bins of x
# shape of x = (n_lags,spect_height) x number of samples
# shape of y = (1, spect_height) x number of samples

n_lags = 2
spect_height = 32
NIN = n_lags*spect_height

In [5]:
# location
datafolder = '/home/AD/kachiem/memmap/memmap_dataset_stimulus/'
visfolder = '/home/AD/kachiem/memmap/memmap_dataset_stimulus_vis/'
x_loc = '%sx_lag%03d.dat' % (datafolder, n_lags)
y_loc = '%sy_lag%03d.dat' % (datafolder, n_lags)
x_loc_vis = '%sx_lag%03d.dat' % (visfolder, n_lags)
y_loc_vis = '%sy_lag%03d.dat' % (visfolder, n_lags)

In [6]:
# get train data
y = np.memmap(x_loc, dtype='float32', mode='r')
num_data_samples = int(len(y) / spect_height)
x_data = np.memmap(x_loc, dtype='float32', mode='r+', shape=(n_lags*spect_height, num_data_samples))
y_data = np.memmap(y_loc, dtype='float32', mode='r+', shape=(spect_height, num_data_samples))

x_data.shape, y_data.shape

((64, 9597568), (32, 9597568))

srihita's spectrogram functions

In [9]:
class Shuffled_memmap_dataset:
    def __init__(self, xfile, yfile, xfile_vis, yfile_vis, vis_ratio=0, valid_ratio=.06):
        y_data = np.memmap(yfile, dtype='float32', mode='r')
        self.num_data_samples = int(len(y_data) / spect_height)
       
        y_data_vis = np.memmap(yfile_vis, dtype='float32', mode='r')
        self.num_data_samples_vis = int(len(y_data_vis) / spect_height)
       
        self.x_data = np.memmap(xfile, dtype='float32', mode='r', shape=(n_lags*spect_height, self.num_data_samples))
        self.y_data = np.memmap(yfile, dtype='float32', mode='r', shape=(spect_height, self.num_data_samples))
       
        self.x_data_vis = np.memmap(xfile_vis, dtype='float32', mode='r', shape=(n_lags*spect_height, self.num_data_samples_vis))
        self.y_data_vis = np.memmap(yfile_vis, dtype='float32', mode='r', shape=(spect_height, self.num_data_samples_vis))
               
        #self.num_vis_samples = int(self.num_data_samples*vis_ratio)
        self.num_valid_samples = int(self.num_data_samples*valid_ratio)
        #vis_start = np.random.randint(self.num_data_samples - self.num_vis_samples)
       
        #self.vis_idxs = np.arange(vis_start, vis_start + self.num_vis_samples)
        #self.data_idxs = np.delete(np.arange(self.num_data_samples), self.vis_idxs)
        self.data_idxs_vis = np.arange(self.num_data_samples_vis)
        self.data_idxs = np.arange(self.num_data_samples)
        #self.num_data_samples -= self.num_vis_samples
       
        np.random.shuffle(self.data_idxs)
        self.valid_idxs, self.data_idxs = np.split(self.data_idxs, [self.num_valid_samples])
        self.num_data_samples -= self.num_valid_samples
           
    def get_data(self, idx):
        return self.x_data[:, idx].T, self.y_data[:, idx].T
   
    def data_iterator(self, batch_size=64):
        np.random.shuffle(self.data_idxs)
        for batch_idx in range(0, self.num_data_samples, batch_size):
            shuff_idx = self.data_idxs[batch_idx:batch_idx+batch_size]
            yield self.get_data(shuff_idx)
   
    def valid_set(self):
        return self.get_data(self.valid_idxs)
   
    def valid_iterator(self, batch_size=64):
        for batch_idx in range(0, self.num_valid_samples, batch_size):
            valid_idx = self.valid_idxs[batch_idx:batch_idx+batch_size]
            yield self.get_data(valid_idx)
   
    def vis_set(self):
        #return self.get_data(self.vis_idxs)
        return self.get_data(self.data_idxs_vis)
   
    #def vis_iterator(self, start=0, end=None, batch_size=1):
    #    if end is None:
    #        end = self.num_vis_samples
    #    for batch_idx in range(start, end, batch_size):
    #        data_idx = self.vis_idxs[batch_idx:batch_idx+batch_size]
    #        yield self.get_data(data_idx)
           
    def vis_iterator(self, start=0, end=None, batch_size=1):
        if end is None:
            end = self.num_data_samples_vis
        for batch_idx in range(start, end, batch_size):
            data_idx = self.data_idxs_vis[batch_idx:batch_idx+batch_size]
            yield self.get_data(data_idx)

dataset = Shuffled_memmap_dataset(x_loc, y_loc, x_loc_vis, y_loc_vis)

putting it all together - input + spectrogram + train_model

In [10]:
## Get visualization dataset
vis_x, vis_y = dataset.vis_set()
# took a subset of test set for visualization
start = 3000
end = start + 1000
sub_vis_x = vis_x[start:end, :]
sub_vis_y = vis_y[start:end, :]

main( ) in train_model.py

In [11]:
## cpc train_model() params / main()
SPECT_HEIGHT = 2048
epochs = 10
batch_size = 32
output_dir = "main"
code_size = 8
hparams = lspct.features.spectrogram.HPARAMS

#path_list = glob.glob((lspct.paths.WAV_DIR / "*.wav").as_posix())[0:19]    # get list of wavs
#training_path_list, validation_path_list = split_validation(path_list, 0.1) # split train and val


In [14]:
train_model(
    epochs=10,
    batch_size=32,
    output_dir='models/memmap',
    code_size=128,
    lr=1e-3,
    terms=4,
    predict_terms=4,
    image_size=64,
    color=True
)

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 31, 31, 64)        1792      
_________________________________________________________________
batch_normalization_v1 (Batc (None, 31, 31, 64)        256       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 15, 15, 64)        36928     
_________________________________________________________________
batch_normalization_v1_1 (Ba (None, 15, 15, 64)        256       
_________________________________________________________________
leak

AttributeError: 'SortedNumberGenerator' object has no attribute 'shape'